<center><img src=img/MScAI_brand.png width=70%></center>

# Scikit-Learn: Hyperparameter Tuning and Cross-Validation

When approaching an ML problem we often train multiple models. There are at least three possibilities:

* Different models *per se*, e.g. Logistic Regression versus SVM
* Tuning hyperparameter values (aka **model selection**)
* Different features (**feature selection**)

This notebook is about hyperparameter tuning only. We'll cover feature selection in the next. 

### Hyperparameter Tuning with a Train-Test Split

To get started, we'll consider a simple scenario: a single train-test split. How can we carry out hyperparameter tuning using Scikit-Learn?

Let's try the `RandomForestRegressor` which wins a lot of Kaggle-type competitions.

In [2]:
import itertools
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 
import pandas as pd

We'll use the Boston housing dataset like before.

In [4]:
Xy = pd.read_csv("data/boston.csv", index_col=0)
Xy.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [3]:
X = Xy.drop("MEDV", axis=1)
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [4]:
y = Xy["MEDV"]
y.head()

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: MEDV, dtype: float64

As we said, we are going to use a single train-test split, as is already familiar.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y)

Now we are ready to run an experiment. Let's use `itertools.product` as before.

In [6]:
n_estimatorss = [1, 5, 10, 50, 100, 500, 1000]
max_depths = [2, 4, 6, 8, 10, None] # NB None
for n_estimators, max_depth in (
    itertools.product(n_estimatorss, max_depths)):
    
    rf = RandomForestRegressor(
        n_estimators=n_estimators, 
        max_depth=max_depth)
    rf.fit(X_train, y_train)
    fmt = "%d\t%s\t%.2f"
    print(fmt % (n_estimators, str(max_depth), 
                     rf.score(X_test, y_test)))

1	2	0.67
1	4	0.84
1	6	0.84
1	8	0.79
1	10	0.80
1	None	0.79
5	2	0.75
5	4	0.86
5	6	0.89
5	8	0.89
5	10	0.89
5	None	0.89
10	2	0.73
10	4	0.87
10	6	0.90
10	8	0.90
10	10	0.89
10	None	0.90
50	2	0.79
50	4	0.88
50	6	0.90
50	8	0.92
50	10	0.92
50	None	0.91
100	2	0.77
100	4	0.88
100	6	0.91
100	8	0.91
100	10	0.92
100	None	0.92
500	2	0.78
500	4	0.89
500	6	0.91
500	8	0.91
500	10	0.92
500	None	0.92
1000	2	0.78
1000	4	0.88
1000	6	0.91
1000	8	0.91
1000	10	0.92
1000	None	0.92


Remember that by default the `score` for regression is the coefficient of determination $R^2$, where higher is better. The best I observe is $R^2 = 0.91$ with e.g. `(50, 8)`.

### Cross-Validation

Disadvantages of a single train-test split:

* Vulnerable to a single random decision (e.g. many "easy" examples in the test set)
* Some of the data doesn't contribute to training

Cross-validation solves these problems by splitting the data into $k$ *folds*, and then training $k$ times, each time on $1-1/k$ of the data, and validating on the remaining $1/k$:

<center><img src=img/grid_search_cross_validation.png width=40%></centre>

<font size=1>https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation</font>

Scikit-Learn provides easy interfaces for cross-validation.

In [7]:
from sklearn.model_selection import cross_val_score
rf = RandomForestRegressor(n_estimators=50, 
                           max_depth=8)
cross_val_score(rf, X_train, y_train, cv=5) 

array([0.82305379, 0.74622642, 0.83550338, 0.8830016 , 0.8856796 ])

Here, we see a *big* difference in performance due to different folds. This is a warning not to blindly trust the result of any single test set.

Next, we can use CV to help tune hyperparameters.

In [8]:
n_estimatorss = [5, 50] # to speed things up
max_depths = [2, 8, None] # we'll try fewer values
for n_estimators, max_depth in (
    itertools.product(n_estimatorss, max_depths)):
    rf = RandomForestRegressor(
        n_estimators=n_estimators, 
        max_depth=max_depth)
    # NB mean
    score = cross_val_score(rf, X_train, y_train, 
                            cv=5).mean() 
    fmt = "%d\t%s\t%.2f"
    print(fmt % (n_estimators, str(max_depth), score))

5	2	0.71
5	8	0.80
5	None	0.76
50	2	0.74
50	8	0.83
50	None	0.83


### Grid Search

What we have done here, and earlier, is a *grid search*: we tried out all combinations of hyperparameter values. We did it manually, but let's never do that again: Scikit-Learn provides it for us. We provide a `dict` giving the values to be tried.

In [9]:
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [5, 50],
              'max_depth': [2, 8, None]}
grid = GridSearchCV(RandomForestRegressor(), 
                    param_grid, cv=5) 

In [10]:
grid.fit(X_train, y_train)
grid.best_params_

/Users/jmmcd/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'max_depth': None, 'n_estimators': 5}

### Multiple `score` functions

There is also the `cross_validate` function which allows us to measure multiple `score` functions and also timing information.

In [19]:
from sklearn.model_selection import cross_validate
scoring = ['r2', 'neg_mean_squared_error']
scores = cross_validate(rf, X_train, y_train, 
                        scoring=scoring, cv=5)
for key in scores:
    print("%s %.2f" % (key, scores[key].mean()))

fit_time 0.05
score_time 0.01
test_r2 0.85
test_neg_mean_squared_error -12.43


### Leave-one-out Cross-Validation

There is even *leave-one-out* cross-validation, where the number of folds equals the number of training samples. However, it doesn't work when `score` is $R^2$! 

```python
from sklearn.model_selection import LeaveOneOut
rf = RandomForestRegressor(n_estimators=50, max_depth=8)
cross_val_score(rf, X, y, cv=LeaveOneOut())
```

We'll see a warning:

```
UndefinedMetricWarning: R^2 score is not well-defined with less than two samples
```

This warning is correct. When running *leave-one-out*, we only get a single $\hat{y}$ after each `fit`. Scikit-Learn tries to calculate $R^2$ on it, and it isn't well-defined. It would be better if Scikit-Learn would just save all the $\hat{y}$ values and then calculate a single $R^2$. Probably they'll fix this soon. But to demonstrate, we could just use MSE instead.

In [14]:
from sklearn.model_selection import LeaveOneOut
rf = RandomForestRegressor(n_estimators=50, max_depth=8)
scores = cross_val_score(rf, X, y, cv=LeaveOneOut(), 
            scoring="neg_mean_squared_error")
print(scores.mean())

-10.76049920030379


There are many more possibilities for model selection, including *stratified* CV:

* https://scikit-learn.org/stable/model_selection.html

But usually just choosing (say) a simple 5-fold CV in a grid search is good enough.